**Data Science for Good:City of Los Angles Job Bulletins Analysis**

**Kernel Content**<br>
1. Objective<br>
2. Problem Statement<br>
3. Data Dictionary<br>
4. Single Structured CSV File Code<br>
5. Data Analysis<br>

**1. Objective**<br>
Using text analysis,The Los Angeles Mayor’s Office wants to improve the city's job bulletins (683 in total) by using text analysis to make suggestions for improvements.


**2. Problem Statement**<br>
The goal is to convert a folder full of plain-text job postings into a single structured CSV file and then to use this data to:<br> (1) identify language that can negatively bias the pool of applicants<br> 
(2) improve the diversity and quality of the applicant pool<br>
(3) make it easier to determine which promotions are available to employees in each job class. (optional)

**3. Data Dictionary**

Import all the needed libraries.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re #to use regular expressions to find text
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data_dictionary = pd.DataFrame({'Field Name':["FILE_NAME","JOB_CLASS_TITLE","JOB_CLASS_NO","REQUIREMENT_SET_ID", "JOB_DUTIES", "EDUCATION_YEARS","SCHOOL_TYPE",
                                            "EDUCATION_MAJOR", "EXPERIENCE_LENGTH", "FULL_TIME_PART_TIME", "EXP_JOB_CLASS_TITLE","EXP_JOB_CLASS_FUNCTION",
                                            "COURSES", "COURSE_LENGTH", "DRIVERS_LICENSE_REQ","DRIV_LIC_TYPE", "ADDTL_LIC", "EXAM_TYPE", "ENTRY_SALARY_GEN",
                                            "ENTRY_SALARY_DWP", "OPEN_DATE"],
                             })

data_dictionary['Description'] = ["The file name of the job bulletin from which each record came.",
                                  "The title of the particular class (e.g., Systems Analyst, Carpenter).",
                                  "The 4-digit code that corresponds to the job class.",
                                  "A/The requirement(s) that has/have or might has/have to be met. These are identified with numbers and letters in the job bulletin.",
                                  "A summary of what someone does in the particular job.",
                                  "Years required in a particular education program",
                                  "School type required (e.g. college or university, high school, certificate, apprenticeship)",
                                  "Major or concentration required",
                                  "Years required in a particular job class or external role.",
                                  "Whether the required experience is full-time or part-time",
                                  "Job class title of Internal City job that one must hold to satisfy a requirement (e.g. Management Assistant)",
                                  "Job experience of jobs (e.g. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases)",
                                  "The necessary subject(s) of required courses",
                                  "The semester or quarter unit count needed.",
                                  "Whether a driverâ€™s license is required, possibly required, or not required. P (or possible) refers to any class that says \"Some positions may require a valid California driver's license\". R (or required) explicitly says a driver's license is required",
                                  "The specific driverâ€™s license type required, Class A, Class B or Class C",
                                  "Additional licenses specified in the job bulletin other than Driver's License",
                                  "Classifications have different rules around whether one has to already have Civil Service status to apply or whether experience has to be in a particular department.",
                                  "The base overall salary range. It captures the first range. Some don't have a range.",
                                  "For job classes that are available in the City at large AND the DWP agency specifically, a second DWP salary range will be provided. This field will capture the first DWP-specific salary range.",
                                  "The date the job bulletin opened"
                               ]

data_dictionary['Notes'] = ["It is the text file name. It consists of the title of the job, code of the job and date in which the text file was created.",
                            "Extracted from file name by searching for the letters in the file name.",
                            "Searched for keywords: \"Class Code:\" and saved four digits that are after it.",
                            "Searched for keyword: \"require\" (verb to avoid confusion between requirements and requirement) and counted the lines below for lines that start with a digit or a letter",
                            "Searched for keword:\"duties\" and saved the lines under it till this a break line",
                            "Searched for keywords:\"one...ten year school_type\" I limited the search to only years",
                            "Searched for keywords:\"college or university, high school, certificate, apprenticeship",
                            "Searched for keywords:\"major in x | apprenticeship as x | college or university in x\"",
                            "Searched for keywords:\"full-time, part-time and apprenticeship as and saved what is before it\"",
                            "Searched for keywords:\"full-time and part-time",
                            "Searched for keywords:\"experience in, experience as, experience with or experience at\" and saved what as after it",
                            "Saved what is after title of the job of past exprience when I found it",
                            "When school_type is college or university, searched for keywords:\"courses\" and saved what is after it till a new line",
                            "When courses are found, I searched for keywords:\"quarter uints and semesters and saved digits before them\"",
                            "Searched for keywords:\"driver's license\" then searched for keywords:\"is required, may require\" to classify it",
                            "After ensuring that a license is required, I searched for keyword:\"class followed by A,B or C\" and saved the character",
                            "Searched for keyword:\"license\" and made sure it is not driver's and saved what is before the word license",
                            "Searcged for keywords:\"OPEN COMPETITIVE BASIS|INTERDEPARTMENTAL PROMOTIONAL|DEPARTMENTAL PROMOTIONAL BASIS\"",
                            "Searched for $ followed by digit. Then searched for word flat to know if it is a range or not",
                            "After finding the $ and digits, I searched for the word dwp to check if there is another salary or not",
                            "Searched for keywords open date: and saved what is after it"
                           ]

data_dictionary['Data Type'] = ['string',
                               'string',
                               'string',
                               'integer',
                               'string',
                               'integer',
                               'string',
                               'string',
                               'integer',
                               'string',
                               'string',
                               'string',
                               'string',
                               'string',
                               'string',
                               'string',
                               'string',
                               'string',
                               'string',
                               'string',
                               'string']

data_dictionary['Accepts Null Values?']=['No',
                                        'No',
                                        'No',
                                        'No',
                                        'No',
                                        'Yes',
                                        'Yes',
                                        'Yes',
                                        'Yes',
                                        'Yes',
                                        'Yes',
                                        'Yes',
                                        'Yes',
                                        'Yes',
                                        'Yes',
                                        'Yes',
                                        'Yes',
                                        'No',
                                        'No',
                                        'Yes',
                                        'No']
data_dictionary

Create CSV file.

In [ ]:
filename = 'Dictionary.csv'

data_dictionary.to_csv(filename,index=False)

print('Saved file: ' + filename)

**4. Single Structured CSV File Code<br>**
Convert a folder full of plain-text job postings into a single structured CSV file.

For starters, I forked the code in Data Science for Good City of LA Starter Kernel, and then I started working from there. Thank you. It was the push that I needed to start.

Overall, the code open text files one by one and start going through each text file line by line. 
In the excel sheet "Kaggle_data_dictionary.csv", the field names and descriptions of each field are mentioned there. There are **21** fields.*FILE_NAME* and *OPEN_DATE* fields are already extracted in the code that I forked.<br><br> 

**For easier computatioins, I made sure to make strings all the same case when I compare them or search for them.**


In [ ]:
def get_drivers_license_req(line):
    drivers_license = [] #list to store the extracted data (req and class).
    drivers_license_class = ""
    additional_license = ""
    #keywords
    words_list1 = "driver's license"
    words_list2 = "is required"
    words_list3 = "may require"
    
    drivers_license_req = ""
    #check if keywords xist
    if words_list1 in line:
        if words_list2 in line: 
            drivers_license_req = "R"
                
        if words_list3 in line:
            drivers_license_req = "P"
    else:
        drivers_license_req = ""
    
    #find class of the license
    drivers_license_class = ""
    drivers_license_class_results = re.findall("Class [ABC] or Class [ABC] driver's license | Class [ABC]", line) #reg expression

    if drivers_license_class_results: #acccording to the dictionary, sometime the class of the license isn't mentioned so i have to check if 
        #a class/es is/are found
        for i in range(len(drivers_license_class_results)):
            if len(drivers_license_class_results) > 1:
                drivers_license_class = drivers_license_class + drivers_license_class_results[i]
            else:
                drivers_license_class = drivers_license_class_results[i]
    
    #format the results
    drivers_license_class = drivers_license_class.replace(" Class ","") #remove word class
    drivers_license_class = list(set(drivers_license_class)) #get unique values 
    
    #store the data
    drivers_license.append(drivers_license_req)
    drivers_license.append(drivers_license_class)
    
    return drivers_license

To get two fields of salary,*ENTRY_SALARY_GEN* and *ENTRY_SALARY_DWP*, I scaned the lines for format $xx,xxx/$xxx,xxx and filtered the results according to **4 salary types**:<br> 
1) flat-rated and not Department of Water and Power(gen)<br>
2) not flat-rated and not Department of Water and Power(gen)<br>
3) flat-rated and Department of Water and Power<br>
4) not flat-rated and Department of Water and Power<br><br>

In [ ]:
def get_salary(line):
    salary_gen = ""
    salary_dwp = "" 
    salary = []
    
    if "flat" in line.lower().strip() and "Department of Water and Power" not in line.strip(): #flat and not dwp(gen)
        salary_gen = re.search(r"\$\d{2,3}\,\d{3}", line.lower()).group()
        salary_dwp = ""
  
    if "flat" not in line.lower().strip() and "Department of Water and Power" not in line.strip(): #not flat and not dwp(gen)
        salary_gen = re.search(r"\$\d{2,3}\,\d{3}\sto\s\$\d{2,3}\,\d{3}|\$\d{2,3}\,\d{3}[*]\sto\s\$\d{2,3}\,\d{3}|\$\d{2,3}\,\d{3}", line.lower()).group().replace("to","-")
        salary_dwp = ""
   
    if "flat" in line.lower().strip() and "Department of Water and Power" in line.strip(): #flat and dwp
        salary_dwp = re.search(r"\$\d{2,3}\,\d{3}", line.lower()).group()
 
    if "flat" not in line.lower().strip() and "Department of Water and Power" in line.strip(): #not flat and dwp
        salary_dwp = re.search(r"\$\d{2,3}\,\d{3}\sto\s\$\d{2,3}\,\d{3}|\$\d{2,3}\,\d{3}\sto\s\$\d{2,3}\,\s\d{3}", line.lower()).group().replace("to","-")
             
    salary.append(salary_gen)
    salary.append(salary_dwp)

    return salary

In [ ]:
def convert_to_integer(text):
    global flag
    flag = 0
    if "one" in text.lower():
        text = "1"
        flag = 1
    if "two" in text.lower():
        text = "2"
        flag = 1
    if "three" in text.lower():
        text = "3"
        flag = 1
    if "four" in text.lower():
        text = "4"
        flag = 1
    if "five" in text.lower():
        text = "5"
        flag = 1
    if "six" in text.lower():
        text = "6"
        flag = 1
    if "seven" in text.lower():
        text = "7"
        flag = 1
    if "eight" in text.lower():
        text = "8"
        flag = 1
    if "nine" in text.lower():
        text = "9"
        flag = 1
    if "ten" in text.lower():
        text = "10"
        flag = 1
    return text

In [ ]:
def get_experience_length(text):
    if re.search(r"\S{1,}\s*\S{1,}\s*of\s*full\s*-\s*time | \S{1,}\s*\S{1,}\s*of\s*part\s*-\s*time | S{1,}\s*\S{1,}\s*apprenticeship\s*as", text, re.IGNORECASE):
        experience_length = re.search(r"\S{1,}\s*\S{1,}\s*of\s*full\s*-\s*time | \S{1,}\s*\S{1,}\s*of\s*part\s*-\s*time | S{1,}\s*\S{1,}\s*apprenticeship\s*as", text, re.IGNORECASE).group()
    else:
        experience_length = ""
    
    if "months" in experience_length.lower():
        experience_length = experience_length.replace("(","")
        experience_length = experience_length.replace(")","")
        experience_length = convert_to_integer(experience_length.lower())
        if flag == 1:
            experience_length = int(experience_length) / 12.0 #convert to years
        elif flag == 0:
            experience_length = experience_length[0:1].strip()
            experience_length = int(experience_length) / 12.0 #convert to years
    else:
        experience_length = convert_to_integer(experience_length.lower())     
    return experience_length

In [ ]:
def get_full_time_part_time(text):
    try:    
        full_time_part_time = re.search(r"full\s*-\s*time | part\s*-\s*time", text).group()
        get_exp_job_class_title(text)

    except: #type isn't full-time or part-time
        full_time_part_time = ""
   
    return full_time_part_time

In [ ]:
def get_exp_job_class_title(text):
    try:
        job = re.search(r"experience\s*in\s*.* | experience\s*as\s*.* | experience\s*with\s*.* | experience\s*at\s*.*", text, re.IGNORECASE).group()
        get_exp_duties(text)
    except:
        job = ""
        
    if "experience in" in job.lower():
        job = job.replace("experience in","")
    if "experience as" in job.lower():
        job = job.replace("experience as","")
    if "experience with" in job.lower():
        job = job.replace("experience with","")
    if "experience at" in job.lower():
        job = job.replace("experience at","")
    return job

In [ ]:
def get_exp_duties(text):
    if re.search(r"[a-zA-Z]{1,}ing\s*.*", text, re.IGNORECASE):
        exp_duties = re.search(r"[a-zA-Z]{1,}ing\s*.*", text, re.IGNORECASE).group()
    else:
        exp_duties = "" 
    return exp_duties

In [ ]:
def get_education_years(text, school_type): 
    found_education_years = ""
    pattern1 = r"(one|two|three|four|five|six|seven|eight|nine|ten)\s*-\s*year\s*(?="+school_type.strip()+")\s*"

    if school_type != "":
        if re.search(pattern1, text, re.IGNORECASE):
            found_education_years = re.search(pattern1, text, re.IGNORECASE).group()
            found_education_years = found_education_years.replace("-year","")
            found_education_years = int(convert_to_integer(found_education_years.lower())) 
        else:
            found_education_years = ""
         
    return found_education_years

In [ ]:
def get_courses(text):
    print("in")
    global courses
    courses = []
    flag = 0
    
    #get the names of the courses
    if re.search(r"courses", text, re.IGNORECASE):
        found_courses = re.search(r"courses\s*.*", text, re.IGNORECASE).group()
        courses.append(found_courses)
    else:
        found_courses = ""
        courses.append(found_courses)
    
    #get number of semesters
    if re.search(r"\d{1,}\s*semester", text, re.IGNORECASE):
        semesters = re.search(r"\d{1,}\s*semester", text, re.IGNORECASE).group()
        flag = 1
    else:
        semesters = ""
    
    #get number of quarter units
    if re.search(r"\d{1,}\s*quarter\s*units", text, re.IGNORECASE):
        units = re.search(r"\d{1,}\s*quarter\s*units", text, re.IGNORECASE).group()
        flag = 2
    else:
        units = ""
    
    #found semesters and quarter units numbers
    if flag == 2:
        semesters_units = semesters + " and " + units
    else:
        semesters_units = ""
    flag = 0
    courses.append(semesters_units)
    return courses

In [ ]:
def get_school_type(text):
    if re.search(r"COLLEGE\s*OR\s*UNIVERSITY | HIGH\s*SCHOOL | APPRENTICESHIP | CERTIFICATE | Semester | Semesters | units", text, re.IGNORECASE):
        school_type = re.search(r"COLLEGE\s*OR\s*UNIVERSITY | HIGH\s*SCHOOL | APPRENTICESHIP | CERTIFICATE | Semester | Semesters | units", text, re.IGNORECASE).group()
        education_years = get_education_years(text, school_type)       
    else: 
        school_type = ""
    
    if re.search(r"COLLEGE\s*OR\s*UNIVERSITY | Semester | Semesters | units", school_type, re.IGNORECASE):
        school_type = "College or University"
    if re.search(r"HIGH\s*SCHOOL", school_type, re.IGNORECASE):
        school_type = "High School"
    if re.search(r"APPRENTICESHIP", school_type, re.IGNORECASE):
        school_type = "Apprenticeship"
    if re.search(r"CERTIFICATE", school_type, re.IGNORECASE):
        school_type = "Certificate"
    
    if "college or university" in school_type.lower(): #get courses' info
        get_courses(text)
    else:
        courses.append("")
        courses.append("")
    return school_type

In [ ]:
def get_education_major(text): # major in x | apprenticeship as x | college or university in x
    education_major = ""
    if re.search(r"major\s*in | apprenticeship\s*as | college\s*or\s*university\s*in", text, re.IGNORECASE):
        education_major = re.search(r"major\s*in\s*.* | apprenticeship\s*as\s*.* | college\s*or\s*university\s*in\s*.*", text, re.IGNORECASE).group()
    else: 
        education_major = ""
        
    if "apprenticeship" in education_major.lower():
        education_major = education_major.replace("apprenticeship as","")
    if "major in" in education_major.lower():
        education_major = education_major.replace("major in","")
    if "college or university in" in education_major.lower():
        education_major = education_major.replace("college or university in","")      
    return education_major

In [ ]:
def get_requirements(text, index):
    requirements = []
    req_count = 0
    req_text = []
    full_time_part_time = ""
    
    if re.match(r"\d{1,}[.]", text[index]): #search for X. where X is number of 1 digit or more at the beginning of the line
        while re.match(r"\w{1,}[.]", text[index]): #number. --->main req & character. --->sub req
            req_count = req_count + 1
            req_text.append(text[index])
            index = index  + 1
        req_text = (" ").join(req_text) #make req_text into a string
      
    else:
        req_count = 1
        req_text.append(text[index])
        req_text = (" ").join(req_text) #make req_text into a string

    shool_type = get_school_type(req_text)
    education_years = get_education_years(req_text, shool_type)
    full_time_part_time = get_full_time_part_time(req_text)
    education_major = get_education_major(req_text)
    experience_length = get_experience_length(req_text)
    experience_job_title = get_exp_job_class_title(req_text)
    experience_job_duties = get_exp_duties(req_text)
    
    requirements.append(req_text)
    requirements.append(req_count)
    requirements.append(education_years)
    requirements.append(shool_type)
    requirements.append(full_time_part_time)
    requirements.append(education_major)
    requirements.append(experience_length)
    requirements.append(experience_job_title)
    requirements.append(experience_job_duties)
    return requirements

In [ ]:
def get_exam_type(text):
    found_exam_type = []
    found = re.findall(r'OPEN COMPETITIVE BASIS|INTERDEPARTMENTAL PROMOTIONAL|DEPARTMENTAL PROMOTIONAL BASIS', text, re.IGNORECASE) #search for these keywords

    if "OPEN COMPETITIVE BASIS" in found:
        found_exam_type.append("OPEN")
       
    if "INTERDEPARTMENTAL PROMOTIONAL" in found:
        found_exam_type.append("INT_DEPT_PROM")
    
    if "DEPARTMENTAL PROMOTIONAL BASIS" in found:
        found_exam_type.append("DEPT_PROM")
        
    found_exam_type = ' ,'.join(found_exam_type)
    return found_exam_type

In [ ]:
def get_add_license(text):
    additional_license = []
    additional_found_license = ""
  
    if re.search(r"(?<!driver's)license\s{1,}(?=as)", text):
        additional_found_license = re.search(r"license\s(?=as)(?=as)\S{1,}\s\S{1,}\s\S{1,}\s\S{1,}\s\S{1,}\s\S{1,}", text).group()

    if re.search(r"\S{1,}\s\S{1,}(?<!driver's)(?<!Class[ABC])(?<!the)(?<!a)(?<!or)(?<!and)\slicense(?!as)", text):
        additional_found_license = re.search(r"\S{1,}\s\S{1,}(?<!driver's)(?<!Class[ABC])(?<!the)(?<!a)(?<!or)(?<!and)\slicense", text).group() 
        
    additional_license.append(additional_found_license)
    
    if len(additional_license) != 0:
        additional_license = ' ,'.join(additional_license)
    else:
        additional_license = ""
    return additional_license

In [ ]:
bulletin_dir = "../input/cityofla/CityofLA/Job Bulletins"
data_list = []

for filename in os.listdir(bulletin_dir):
    with open(bulletin_dir + "/" + filename, 'r', errors='ignore') as f:
        text = f.readlines()
        for line in range(len(text)):
            text[line] = text[line].replace(";","")
            #print(line)
            #parse job bulletins
            job_class_title = re.search(r"\D+",filename).group() #title of the job
            print(job_class_title)
            if "Class Code:" in text[line]: #get class code
                job_class_no = re.search(r"\d{4}",text[line]).group()
            if "REQUIRE" in text[line]: #used part of word requirments since it can be singular or plural (Requirements/Qualifications section)
                requirements = get_requirements(text, line+2)
                req_set_id = requirements[1]
                education_years = requirements[2]
                school_type = requirements[3]
                full_time_part_time = requirements[4]
                education_major = requirements[5]
                experience_length = requirements[6]
                experience_job_title = requirements[7]
                exp_duties = requirements[8]
                found_courses = courses[0]
                semeters_units_found = courses[1]
            if "DUTIES" in text[line]: #get duties by searching for headline "DUTIES" and storing paragraph below it by two lines
                job_duties = text[line+2]
            if "driver's license" in text[line]: #get driver's license information
                drivers_license_req = get_drivers_license_req(text[line])[0]
                if len(get_drivers_license_req(text[line])[1]) > 1:
                    drive_lic_type = '/'.join(get_drivers_license_req(text[line])[1])
                else:
                    drive_lic_type = str((get_drivers_license_req(text[line])[1]))
            
            if "license" in text[line].lower(): #get additional licesnse if exist
                if "driver's license" not in text[line].lower(): #check first it not a driving license
                    add_license = get_add_license(text[line])
                else:
                    add_license = ""
               
            if "THIS EXAMINATION IS TO BE GIVEN" in text[line].upper():#get exam type
                exam_type = get_exam_type(text[line+1])
            if re.search(r"\$\d{2,3}\,\d{3}", text[line]): #get salary and salary dwp(if it exists). search for $xx,xxx/$xxx,xxx expression
                salary_gen = get_salary(text[line])[0] #entry salary
                salary_dwp = get_salary(text[line])[1] #dwp salary
            if "Open Date:" in text[line]: #get open date
                job_bulletin_date = text[line].split("Open Date:")[1].split("(")[0].strip()
                
        data_list.append([filename, job_class_title, job_class_no, req_set_id, job_duties, education_years,
                          school_type, education_major, experience_length, full_time_part_time, experience_job_title,
                          exp_duties, found_courses, semeters_units_found, drivers_license_req, 
                          drive_lic_type, add_license, exam_type, salary_gen, salary_dwp, job_bulletin_date])
        entry_salary_dwp = "0"

In [ ]:
df = pd.DataFrame(data_list) #convert the extracted data to dataframe

In [ ]:
#add row of field names to the dataframe
df.columns = ["FILE_NAME","JOB_CLASS_TITLE","JOB_CLASS_NO","REQUIREMENT_SET_ID", "JOB_DUTIES", "EDUCATION_YEARS",
              "SCHOOL_TYPE", "EDUCATION_MAJOR", "EXPERIENCE_LENGTH", "FULL_TIME_PART_TIME", "EXP_JOB_CLASS_TITLE",
              "EXP_JOB_CLASS_FUNCTION","COURSES", "COURSE_LENGTH", "DRIVERS_LICENSE_REQ",
              "DRIV_LIC_TYPE", "ADDTL_LIC", "EXAM_TYPE", "ENTRY_SALARY_GEN", "ENTRY_SALARY_DWP", "OPEN_DATE"] 

In [ ]:
df.head()

**5. Data Analysis **(under progress)

**5.1 Salary Analysis**<br>
1. Format salaries.
Note: I only took the starting salary of jobs that have range for salary. For example: salary is \$89,000 - \$100,000, I considered only \$89,000

In [ ]:
salaries = []
#format ENTRY_SALARY_GEN by removing $" , "," and "-" and convert it to int
df['ENTRY_SALARY_GEN'] = [s.replace("$","") for s in df['ENTRY_SALARY_GEN']]
df['ENTRY_SALARY_GEN'] = [s.replace(" ","") for s in df['ENTRY_SALARY_GEN']]
df['ENTRY_SALARY_GEN'] = [s.replace(",","") for s in df['ENTRY_SALARY_GEN']]
df['ENTRY_SALARY_GEN'] = [s.replace("*","") for s in df['ENTRY_SALARY_GEN']] #there is one salary that had "*"so I had to manually remove it
df['ENTRY_SALARY_GEN'] = [s.split("-") for s in df['ENTRY_SALARY_GEN']]

for s in df['ENTRY_SALARY_GEN']:
    if s[0] != "":
        salaries.append(int(s[0]))

2. plot salaries and check the distrubution of salaries.

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(salaries,kde=False)
plt.title('Salary Distribution')
plt.show()

3. analyze the graph.<br>
Among all salaries, the **MIN salary** is around **\$30,000** ,and the **MAX salary** is around **\$230,000**.<br>
Most jobs have a salary that is around **\$80,000**(avergae)<br>

4. Suggestions for Improvements<br>
According to Gobankingrates.com, the income needed for homeowners is **\$150,392** and for **renters is \$136,208**. Accroding to the graph above, the average salary is **\$80,000**.<br> There are **\$70,392 shortage for homeowners** and **\$56,208 shortage for renters**. A review on suggested salaries to increase them will most likely increase the quality of the pool of the applicants. If there is a rewarding pay to live comfortably in LA, the jobs will be more appealing.

**5.2 Number of Requirements Analysis**<br>


1. Plot the number of requirements and check the distribution.

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(df.REQUIREMENT_SET_ID,kde=False)
plt.title('Number of Requirements Distribution')
plt.show()
#print(df.REQUIREMENT_SET_ID.to_string(index=True))

In [ ]:
#calculate the mean of the number of the requirements
print(df.REQUIREMENT_SET_ID.mean())

2. Analyze the graph.<br>
Most of the jobs have around 1 to 2 requiremets which give chance to more people.


**5.3 School Type Analysis**<br>


1. Plot the data.

In [ ]:
df['SCHOOL_TYPE'].value_counts() #group school_type by type

**Note:** I won't consider the 477 unknown school type in the pie chart since the reason behind them missing is unknown. It can be because the patterns I used to search for school type didn't catch them or they might be not mentioned. I didn't have the time to check which reason it is.

In [ ]:
labels = 'College or University', 'Certificate', 'Apprenticeship', 'High School'
sizes = [99, 44, 37, 11]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.0f%%',shadow=True, startangle=140)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

2. Analyze the pie chart.<br>
52% (more than half of the jobs) require a college or university degree. While this might ensure that qualified candidtaes only can apply, it can also be a hindrance for the diversity of the pool of the applicants. Due to high costs of colleges, they are not the first option after high school. I suggest that the jobs to be waivered and give more room to those who don't have a college or university degree while ask for extra certificates that are cheaper than tuition of the colleges and universities for example.

Thank you for checking my humble notebook! :D 